In [1]:
import pandas as pd
import numpy as np

In [2]:
from rating_to_category import rating_to_number
from chessPreprocessor.preprocessor import Preprocessor
import chess.pgn

Попробуем собрать статистику по 1 игре

In [3]:
proc_first = Preprocessor()
required_headers = ['Event', 'Result', 'WhiteElo', 'BlackElo', 'WhiteRatingDiff', 'BlackRatingDiff', 'ECO', 'Opening',
                    'TimeControl', 'Termination']
with open("../data/lichess_db_standard_rated_2013-01.pgn") as pgn:
    while True:
        game = chess.pgn.read_game(pgn)
        if game is None:
            break
        if not all(header in game.headers for header in required_headers) or '?' in game.headers['Event'] + \
                game.headers['Result'] + game.headers['WhiteElo'] + game.headers['BlackElo'] + game.headers[
            'WhiteRatingDiff'] + game.headers['BlackRatingDiff'] + game.headers['ECO'] + game.headers[
            'Opening'] + game.headers['TimeControl'] + game.headers['Termination']:
            continue
        proc_first.read_pgn_from_string(str(game.mainline_moves()))
        break

Засечем время, за которое мы получаем статистику по 1 игре

In [4]:
%%time
proc_first.calculate_wdl()
proc_first.calculate_evaluation_stat()
proc_first.calculate_n_best_lines(n=3)

CPU times: user 59 ms, sys: 15.8 ms, total: 74.8 ms
Wall time: 57.2 s


Видим, что если мы захотим собрать статистику по большому датасету, то ударимся об то, что это будет НУ ОЧЕНЬ долго. Давайте попробуем ускорить этот процесс (начнем с разных процессов). Для начала поймем в каком формате хотим хранить данные. Давайте запоминать id игры, номер хода и сохранять всю имеющуюся статистику. Также будем хранить id для наших игр. Таким образом создадим свзять между датасетами

In [5]:
df_stat_first = proc_first.get_stats_per_move(add_wdl_stats=True, add_evaluation_stats=True, n_best_lines=3)
df_stat_first['game_id'] = 0
df_stat_first['move_number'] = np.arange(df_stat_first['game_id'].shape[0])
df_stat_first.head(3)

,move,win,draw,lose,centipawns,moves_to_force_mate,best_line_1_move,best_line_1_centipawns,best_line_1_moves_to_force_mate,best_line_2_move,best_line_2_centipawns,best_line_2_moves_to_force_mate,best_line_3_move,best_line_3_centipawns,best_line_3_moves_to_force_mate,game_id,move_number
0,e2e4,0.020,0.900,0.080,33.0,NaN,e2e4,44.0,NaN,d2d4,33,None,g1f3,24,None,0,0
1,e7e6,0.093,0.890,0.017,41.0,NaN,c7c5,32.0,NaN,e7e5,43,None,e7e6,52,None,0,1
2,d2d4,0.014,0.878,0.108,49.0,NaN,d2d4,27.0,NaN,b1c3,27,None,g1f3,15,None,0,2


Отлично! Такой формат нас более чем устроит. Запомним его, дальше он нам пригодится

Но вот незадача, мы не знаем размеры нашего датафрейма. Давайте посмотрим на распределение классов. Хорошо что мы уже получали весь датафрейм, воспользуемся им

In [6]:
df = pd.read_csv('../data/lichess_db_standard_rated_2013-01.csv')
df['white_rating_num'] = df['white_elo'].apply(rating_to_number)
df['black_rating_num'] = df['black_elo'].apply(rating_to_number)
df['white_rating_num'].value_counts()

white_rating_num
2    46018
3    35050
1    13483
0    12367
4     8170
5     3716
6     2300
7       10
Name: count, dtype: int64

In [7]:
df['black_rating_num'].value_counts()

black_rating_num
2    46658
3    32426
1    14139
0    13864
4     7988
5     3657
6     2375
7        7
Name: count, dtype: int64

Мы знаем, что одна игра обрабатывается ~1 минуту. Если мы спим ~6 часов, тогда нам хватит на 360 игр! Тогда давайте возьмем с каждого класса ~500 игр, но т.к. в игре могут играть разные классы, то ~1000 участников с каждым рейтингом. Тогда посчитаем количество игр, которые будут в итоговом датасете

In [8]:
proc = Preprocessor()
MAX_USER_NUM = 1000
rating_num_count = {0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0}

i = 0
required_headers = ['Event', 'Result', 'WhiteElo', 'BlackElo', 'WhiteRatingDiff', 'BlackRatingDiff', 'ECO', 'Opening',
                    'TimeControl', 'Termination']
with open("../data/lichess_db_standard_rated_2013-01.pgn") as pgn:
    while True:
        game = chess.pgn.read_game(pgn)
        if game is None:
            break

        if not all(header in game.headers for header in required_headers) or '?' in game.headers['Event'] + \
                game.headers['Result'] + game.headers['WhiteElo'] + game.headers['BlackElo'] + game.headers[
            'WhiteRatingDiff'] + game.headers['BlackRatingDiff'] + game.headers['ECO'] + game.headers[
            'Opening'] + game.headers['TimeControl'] + game.headers['Termination']:
            continue
        white_elo = int(game.headers['WhiteElo'])
        white_num = rating_to_number(white_elo)

        black_elo = int(game.headers['BlackElo'])
        black_num = rating_to_number(black_elo)
        if rating_num_count[white_num] < MAX_USER_NUM or rating_num_count[black_num] < MAX_USER_NUM:
            rating_num_count[white_num] += 1
            rating_num_count[black_num] += 1
            i += 1
i

4702

Не густо, но будем работать с чем имеем. Если поймем, что данных мало, то на этот случай сохраним индексы игр в датасете, чтобы сохранять игры, на которых мы уже обучились

Теперь самое интересное, давай сохраним в 2 датасета:
1) Информацию о всей партии
2) Информацию про ходы для каждой партии
Сохранять мы будем для того, чтобы читать данные из csv, что хотя-бы реально по времени

In [9]:
proc = Preprocessor()
rating_num_count = {0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0}
GAME_COUNT = i

i = 0
game_id = 0
required_headers = ['Event', 'Result', 'WhiteElo', 'BlackElo', 'WhiteRatingDiff', 'BlackRatingDiff', 'ECO', 'Opening',
                    'TimeControl', 'Termination']

events = np.empty(GAME_COUNT, dtype=object)
results = np.empty(GAME_COUNT, dtype=object)

white_elo = np.empty(GAME_COUNT, dtype=int)
black_elo = np.empty(GAME_COUNT, dtype=int)
white_rating_diff = np.empty(GAME_COUNT, dtype=int)
black_rating_diff = np.empty(GAME_COUNT, dtype=int)

ecos = np.empty(GAME_COUNT, dtype=object)
openings = np.empty(GAME_COUNT, dtype=object)

time_control = np.empty(GAME_COUNT, dtype=object)
termination = np.empty(GAME_COUNT, dtype=object)

game_ids = np.empty(GAME_COUNT, dtype=int)

all_moves = []

with open("../data/lichess_db_standard_rated_2013-01.pgn") as pgn:
    while True:
        game = chess.pgn.read_game(pgn)
        if game is None:
            break

        if not all(header in game.headers for header in required_headers) or '?' in game.headers['Event'] + \
                game.headers['Result'] + game.headers['WhiteElo'] + game.headers['BlackElo'] + game.headers[
            'WhiteRatingDiff'] + game.headers['BlackRatingDiff'] + game.headers['ECO'] + game.headers[
            'Opening'] + game.headers['TimeControl'] + game.headers['Termination']:
            continue
        game_white_elo = int(game.headers['WhiteElo'])
        white_num = rating_to_number(game_white_elo)

        game_black_elo = int(game.headers['BlackElo'])
        black_num = rating_to_number(game_black_elo)
        if rating_num_count[white_num] < MAX_USER_NUM or rating_num_count[black_num] < MAX_USER_NUM:
            rating_num_count[white_num] += 1
            rating_num_count[black_num] += 1

            events[i] = game.headers['Event']
            results[i] = game.headers['Result']
            white_elo[i] = game_white_elo
            black_elo[i] = game_black_elo
            white_rating_diff[i] = game.headers['WhiteRatingDiff']
            black_rating_diff[i] = game.headers['BlackRatingDiff']
            ecos[i] = game.headers['ECO']
            openings[i] = game.headers['Opening']
            time_control[i] = game.headers['TimeControl']
            termination[i] = game.headers['Termination']
            game_ids[i] = game_id

            all_moves.append(str(game.mainline_moves()))
            i += 1
        game_id += 1

df = pd.DataFrame({'Events': events, 'results': results, 'white_elo': white_elo, 'black_elo': black_elo,
                   'white_rating_diff': white_rating_diff, 'black_rating_diff': black_rating_diff, 'ecos': ecos,
                   'openings': openings, 'time_control': time_control, 'termination': termination,
                   'game_id': game_ids})


In [10]:
df.head(5)

,Events,results,white_elo,black_elo,white_rating_diff,black_rating_diff,ecos,openings,time_control,termination,game_id
0,Rated Classical game,1-0,1639,1403,5,-8,C00,French Defense: Normal Variation,600+8,Normal,0
1,Rated Classical game,1-0,1654,1919,19,-22,D04,"Queen's Pawn Game: Colle System, Anti-Colle",480+2,Normal,1
2,Rated Classical game,1-0,1643,1747,13,-94,C50,Four Knights Game: Italian Variation,420+17,Normal,2
3,Rated Bullet game,0-1,1824,1973,-6,8,B12,Caro-Kann Defense: Goldman Variation,60+1,Normal,3
4,Rated Bullet game,0-1,1765,1815,-9,9,C00,French Defense: La Bourdonnais Variation,60+1,Normal,4


Запишем этот data_frame в csv, чтобы с ним можно было быстрее работать

In [27]:
df.to_csv("../data/clear_data.csv", encoding='utf-8', index=False)

Теперь давайте напишем функцию, которая будет принимать отрезок ходов, которые мы хотим обрабатывать, соответсвующие индексы 

In [13]:
def get_preprocessed_df(l, r, moves, _game_ids):
    # count in [l, r)
    _all_stats = []
    for ind in range(l, r, 1):
        _proc = Preprocessor()
        _proc.read_pgn_from_string(moves[ind])
        _proc.calculate_wdl()
        _proc.calculate_evaluation_stat()
        _proc.calculate_n_best_lines(n=3)
        df_stat = _proc.get_stats_per_move(add_wdl_stats=True, add_evaluation_stats=True, n_best_lines=3)
        df_stat['game_id'] = _game_ids[ind]
        df_stat['move_number'] = np.arange(df_stat['game_id'].shape[0])
        _all_stats.append(df_stat)

    return _all_stats

Давайте проверим, действительно ли с потоками программа будет выполняться быстрее. Для этого засечем время и проверим на отрезке [0, 9)

In [14]:
%%time
all_stats_without_thread = get_preprocessed_df(0, 9, all_moves, game_ids)

CPU times: user 1.21 s, sys: 276 ms, total: 1.49 s
Wall time: 12min 50s


In [16]:
%%time
from concurrent.futures import ThreadPoolExecutor

with ThreadPoolExecutor() as executor:
    future1 = executor.submit(get_preprocessed_df, 0, 3, all_moves, game_ids)
    future2 = executor.submit(get_preprocessed_df, 3, 6, all_moves, game_ids)
    future3 = executor.submit(get_preprocessed_df, 6, 9, all_moves, game_ids)

    return1 = future1.result()
    return2 = future2.result()
    return3 = future3.result()

len(return1 + return2 + return3)

CPU times: user 1.56 s, sys: 372 ms, total: 1.93 s
Wall time: 7min 8s


9

Прирост хоть и не в 3 раза, но какой-то да есть. Давайте попробуем запустить процессы и посмотреть как долго они будут работать. Для них нужна своя фунция, которая будет принимать очередь (с помощью которой умеют общаться потоки)

In [18]:
def get_preprocessed_df_in_process(l, r, moves, _game_ids, return_dict, process_id):
    # count in [l, r)
    _all_stats = []
    for ind in range(l, r, 1):
        _proc = Preprocessor()
        _proc.read_pgn_from_string(moves[ind])
        _proc.calculate_wdl()
        _proc.calculate_evaluation_stat()
        _proc.calculate_n_best_lines(n=3)
        df_stat = _proc.get_stats_per_move(add_wdl_stats=True, add_evaluation_stats=True, n_best_lines=3)
        df_stat['game_id'] = _game_ids[ind]
        df_stat['move_number'] = np.arange(df_stat['game_id'].shape[0])
        _all_stats.append(df_stat)
    return_dict[process_id] = _all_stats

In [20]:
%%time
from multiprocessing import Process, Manager

with Manager() as manager:
    return_dict = manager.dict()
    processes = [
        Process(target=get_preprocessed_df_in_process, args=(i * 3, (i + 1) * 3, all_moves, game_ids, return_dict, i))
        for i in range(3)]
    for p in processes:
        p.start()
    for p in processes:
        p.join()
    print(len(list(return_dict.values())))


3
CPU times: user 32.8 ms, sys: 36 ms, total: 68.9 ms
Wall time: 7min 13s


Процессы нас тоже не спасли (почему-то). Тогда давайте напишем на потоках, прочитаем первые 240 записей из 8 потоков. Для этого сначала проверим что результаты потоки вернули верные (сравним их с результатами 1 потока), а затем поставим так скажем на загрузгу наши потоки

8 потоков обрабатывали (каждый) 1 игру в сумме 4 минуты. Что примерно 30 секунд на игру
16 потоков обрабатывали (каждый) 1 игру в сумме 8 минуты. Что примерно также 30 секунд на игру. Поэтому оставим 8 потоков 

In [31]:
%%time
from concurrent.futures import ThreadPoolExecutor

THREAD_COUNT = 8
STEP = 30
all_res = []
with ThreadPoolExecutor() as executor:
    all_future = [executor.submit(get_preprocessed_df, i * STEP, (i + 1) * STEP, all_moves, game_ids) for i in
                  range(THREAD_COUNT)]
    for future in all_future:
        all_res += future.result()
len(all_res)

CPU times: user 1min 52s, sys: 24.8 s, total: 2min 17s
Wall time: 2h 38min 54s


240

In [32]:
all_res[239]

,move,win,draw,lose,centipawns,moves_to_force_mate,best_line_1_move,best_line_1_centipawns,best_line_1_moves_to_force_mate,best_line_2_move,best_line_2_centipawns,best_line_2_moves_to_force_mate,best_line_3_move,best_line_3_centipawns,best_line_3_moves_to_force_mate,game_id,move_number
0,e2e4,0.020,0.900,0.080,33.0,NaN,e2e4,44.0,NaN,d2d4,33.0,NaN,g1f3,24.0,NaN,239,0
1,e7e5,0.088,0.894,0.018,38.0,NaN,c7c5,32.0,NaN,e7e5,43.0,NaN,e7e6,52.0,NaN,239,1
2,g1f3,0.020,0.900,0.080,33.0,NaN,g1f3,41.0,NaN,f1e2,6.0,NaN,b1c3,0.0,NaN,239,2
3,d7d6,0.154,0.837,0.009,67.0,NaN,b8c6,58.0,NaN,g8f6,76.0,NaN,d7d6,78.0,NaN,239,3
4,g2g3,0.027,0.917,0.056,16.0,NaN,d2d4,71.0,NaN,f1c4,44.0,NaN,b1c3,41.0,NaN,239,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104,g6f7,0.000,0.000,1.000,7163.0,NaN,g6f7,7149.0,NaN,g6f5,5750.0,NaN,g6h5,5652.0,NaN,239,104
105,h8h7,1.000,0.000,0.000,7239.0,NaN,h8h7,7096.0,NaN,h8h7,7096.0,NaN,h8h7,7096.0,NaN,239,105
106,g5g6,0.000,0.000,1.000,NaN,10.0,g5g6,7222.0,NaN,f7e7,5849.0,NaN,f7e6,5849.0,NaN,239,106
107,h7h8,1.000,0.000,0.000,NaN,3.0,h7h6,NaN,11.0,h7h8,NaN,3.0,h7h8,NaN,3.0,239,107


In [33]:
res = pd.concat([i for i in all_res])
res.to_csv("../data/first_240.csv", encoding='utf-8', index=False)